In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '5,6'

from pytorch_lightning import utilities as pl_utils
from pytorch_lightning.trainer.trainer import Trainer
from pytorch_lightning.plugins import DDPPlugin, DataParallelPlugin
import torch, glob
import torch.nn as nn
import itertools
import pathlib, argparse
from functools import partial
from collections import OrderedDict
import numpy as np
import sys

sys.path.append('..')
sys.path.append('../deep-learning-base')

import plot_helper as plt_hp
import output as out
from training import LitProgressBar, NicerModelCheckpointing
import training.finetuning as ft
import architectures as arch
from architectures.callbacks import AdvAttackWrapper
from attack.callbacks import AdvCallback
from datasets.data_modules import DATA_MODULES
import datasets.dataset_metadata as dsmd
from partially_inverted_reps.partial_loss import PartialInversionLoss, PartialInversionRegularizedLoss
from partially_inverted_reps import DATA_PATH_IMAGENET, DATA_PATH, SERVER_PROJECT_PATH
from training.partial_inference_layer import EnsembleHead, HardEnsembleLoss


BASE_DATASET = 'imagenet'
FINETUNING_DATASETS = ['cifar10', 'cifar100', 'flowers', 'oxford-iiit-pets']
MODEL = 'resnet50'
BATCH_SIZE = 100
CHECKPOINT_PATHS = {
    'nonrob': '',
    'robustl2eps3': \
    '/NS/robustness_2/work/vnanda/adv-robustness/logs/robust_imagenet/eps3/resnet-50-l2-eps3.ckpt'
}
APPEND_PATH = 'nonrob'
MODE = 'random'
MODEL_LOSSES = {
    'hard': HardEnsembleLoss(nn.CrossEntropyLoss(reduction='none')),
    'soft': None
}


SEED = 2
NUM_NODES = 1
DEVICES = 2
STRATEGY = DataParallelPlugin() if DEVICES > 1 else None

PARTIAL_CHOICE_SEEDS = range(1,6)

/NS/twitter_archive2/work/vnanda/minconda3/envs/dl_base/lib/python3.7/site-packages/paramiko/transport.py:236: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [ ]:
def generate_ensemble_preds(append_path, ensemble_type):
    dataset_to_clean_accs, dataset_to_rob_accs = {}, {}
    for FINETUNING_DATASET in FINETUNING_DATASETS:
        dm = DATA_MODULES[FINETUNING_DATASET](
            data_dir=DATA_PATH_IMAGENET if 'imagenet' in FINETUNING_DATASET else DATA_PATH,
            transform_train=dsmd.TRAIN_TRANSFORMS_TRANSFER_DEFAULT(224),
            transform_test=dsmd.TEST_TRANSFORMS_DEFAULT(224),
            batch_size=BATCH_SIZE)
        dm.init_remaining_attrs(BASE_DATASET)

        m1 = arch.create_model(MODEL, BASE_DATASET, pretrained=True,
                               checkpoint_path=CHECKPOINT_PATHS[append_path], 
                               seed=SEED, 
                               callback=partial(AdvAttackWrapper,
                                                dataset_name=BASE_DATASET,
                                                return_adv_samples=True,
                                                loss=MODEL_LOSSES[ensemble_type]
                                                )) ## assign mean and std from source dataset
        PARTIAL_FRACTIONS = sorted(
            list(set(
                [float(x.split('/frac-')[1].split('-')[0]) for x in \
                    glob.glob(f'./checkpoints/{MODEL}-base-'
                              f'{BASE_DATASET}-ft-{FINETUNING_DATASET}/'
                              f'*-bs-256')]
            )))
        frac_to_layers = OrderedDict()
        for frac, seed in itertools.product(PARTIAL_FRACTIONS, PARTIAL_CHOICE_SEEDS):
            FINETUNED_CHECKPOINT = glob.glob(
                f'./checkpoints/{MODEL}-base-'
                f'{BASE_DATASET}-ft-{FINETUNING_DATASET}/'
                f'frac-{frac:.5f}-mode-{MODE}-seed-{seed}-lr-0.1-bs-256/'
                f'{APPEND_PATH}/*-topk=1.ckpt')
            if len(FINETUNED_CHECKPOINT) == 0:
                continue
            FINETUNED_CHECKPOINT = FINETUNED_CHECKPOINT[0]
            state_dict = torch.load(FINETUNED_CHECKPOINT)
            new_layer = ft.setup_model_for_finetuning(
                m1.model, 
                dsmd.DATASET_PARAMS[FINETUNING_DATASET]['num_classes'],
                MODE, frac, seed, inplace=False)
            new_layer.load_state_dict({'.'.join(k.split('.')[-2:]):v \
                                        for k,v in state_dict['state_dict'].items()}, strict=True)
            if hasattr(new_layer, 'neuron_indices') and 'neuron_indices' in state_dict:
                assert torch.all(new_layer.neuron_indices == state_dict['neuron_indices'])
            frac_to_layers[frac] = frac_to_layers[frac] + [new_layer] \
                if frac in frac_to_layers else [new_layer]
        
        pl_utils.seed.seed_everything(SEED, workers=True)
        adv_callback = AdvCallback(constraint_train='2',
                               eps_train=3.,
                               step_size=1.,
                               iterations_train=10,
                               iterations_test=100,
                               random_start_train=False,
                               random_restarts_train=0,
                               return_image=True, 
                               do_tqdm=False,
                               custom_loss=MODEL_LOSSES[ensemble_type])
        trainer = Trainer(accelerator='gpu',
                          devices=DEVICES,
                          num_nodes=NUM_NODES,
                          strategy=STRATEGY,
                          log_every_n_steps=1,
                          auto_select_gpus=True,
                          deterministic=True,
                          check_val_every_n_epoch=1,
                          num_sanity_val_steps=0,
                          sync_batchnorm=True,
                          callbacks=[
                            LitProgressBar(['loss',
                                            'running_acc_clean',
                                            'running_acc_adv']),
                            adv_callback])

        frac_to_clean_acc, frac_to_rob_acc = [], []
        name, _ = list(m1.model.named_modules())[-1]
        for frac in PARTIAL_FRACTIONS:
            print (FINETUNING_DATASET, frac)
            m1.model.__setattr__(name, EnsembleHead(frac_to_layers.get(frac), ensemble_type))
            predictions = trainer.predict(m1, dataloaders=[dm.test_dataloader()])
            if trainer.is_global_zero:
                clean_pred = torch.argmax(predictions[0][1], 1)
                adv_pred = torch.argmax(predictions[1][1], 1)
                y = predictions[2]
                frac_to_clean_acc.append(torch.sum(clean_pred == y)/len(y))
                frac_to_rob_acc.append(torch.sum(adv_pred == y)/len(y))
        
        dataset_to_clean_accs[FINETUNING_DATASET] = frac_to_clean_acc
        dataset_to_rob_accs[FINETUNING_DATASET] = frac_to_rob_acc
    return dataset_to_clean_accs, dataset_to_rob_accs

In [ ]:
def plotter(dataset_to_clean_accs, dataset_to_rob_accs, append_path, ensemble_type):
    for FINETUNING_DATASET in FINETUNING_DATASETS:
        plt_str = '<<TableOfContents()>>\n\n'\
            f'== Ensemble Predictions ==\n\n=== {FINETUNING_DATASET} ===\n\n'
        frac_to_clean_acc, frac_to_rob_acc = \
            dataset_to_clean_accs[FINETUNING_DATASET], dataset_to_rob_accs[FINETUNING_DATASET]
        baseline_clean, baseline_rob = frac_to_clean_acc[-1], frac_to_rob_acc[-1]
        x_vals = [PARTIAL_FRACTIONS[idx] for idx in range(len(frac_to_clean_acc[:-1]))]
        plt_str += '{}\n\n'.format(plt_hp.get_wiki_link(plt_hp.line_plot(
            [frac_to_clean_acc[:-1], frac_to_rob_acc[:-1]], 
            'Fraction of neurons', 'Ensemble Accuracy', FINETUNING_DATASET, 
            subfolder=BASE_DATASET, 
            filename=f'{MODEL}_{FINETUNING_DATASET}_{append_path}_{ensemble_type}', 
            extension='png', x_vals=x_vals, 
            legend_vals=['Clean Acc', 'Robust Acc ' + r'($\ell_2 \epsilon = 3$)', 
                'Clean (Full Layer)', 'Rob (Full Layer)'], 
            vertical_line=None, horizontal_lines=[baseline_clean, baseline_rob], 
            colors=[plt_hp.COLORS[0], plt_hp.COLORS[1], plt_hp.COLORS[0], plt_hp.COLORS[1]], 
            linestyles=['-', '-', ':', ':'], y_lims=(0.,1.), root_dir='./partially_inverted_reps', 
            paper_friendly_plots=False, plot_inside=False, legend_location='best', 
            savefig=True, figsize=(10,6), marker=[True, True, False, False], 
            results_subfolder_name='ensemble_analysis', grid_spacing=None, 
            legend_ncol=None), SERVER_PROJECT_PATH, size=700))
        with open(f'partially_inverted_reps/results/ensemble_analysis/{BASE_DATASET}/'
                  f'wiki_results_{FINETUNING_DATASET}-{append_path}-{ensemble_type}.txt', 'w') as fp:
            fp.write(plt_str)
        out.upload_results(['partially_inverted_reps/{}/{}/{}'.format(
            plt_hp.RESULTS_FOLDER_NAME, 
            'ensemble_analysis', BASE_DATASET)], 
            f'partially_inverted_reps/{plt_hp.RESULTS_FOLDER_NAME}', 
            SERVER_PROJECT_PATH, '.png')

In [ ]:
append_path = 'robustl2eps3'
ensemble_type = 'hard'
clean_accs_rob_hard, rob_accs_rob_hard = generate_ensemble_preds(append_path, ensemble_type)
plotter(clean_accs_rob, rob_accs_rob, append_path, ensemble_type)

Global seed set to 2
Global seed set to 2
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
Global seed set to 0


cifar10 0.0005


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [5,6]


Predicting: 0it [00:00, ?it/s]

In [ ]:
append_path = 'robustl2eps3'
ensemble_type = 'soft'
clean_accs_rob_soft, rob_accs_rob_soft = generate_ensemble_preds(append_path, ensemble_type)
plotter(clean_accs_rob, rob_accs_rob, append_path, ensemble_type)